In [15]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
'''
@Project ：PycharmProjects
@File    ：巨潮资讯年报2.0.py
@IDE     ：PyCharm
@Author  ：lingxiaotian
@Date    ：2023/5/20 12:38
'''

import requests
import re
import openpyxl
import time
import os


# 放到桌面上的 cninfo_output 文件夹
output_dir = os.path.expanduser("~/Desktop/cninfo_output")
os.makedirs(output_dir, exist_ok=True)



GZH = "【年报】"
def get_report(page_num, date):
    url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
        "Content-Length": "195",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Host": "www.cninfo.com.cn",
        "Origin": "http://www.cninfo.com.cn",
        "Proxy-Connection": "keep-alive",
        "Referer": "http://www.cninfo.com.cn/new/commonUrl/pageOfSearch?url=disclosure/list/search&checkedCategory=category_ndbg_szsh",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42",
        "X-Requested-With": "XMLHttpRequest"
    }
    data = {
        "pageNum": page_num,
        "pageSize": 30,
        "column": "szse",
        "tabName": "fulltext",
        "plate": plate,
        "searchkey": "",
        "secid": "",
        "category": "category_ndbg_szsh",
        "trade": trade,
        "seDate": date,
        "sortName": "code",
        "sortType": "asc",
        "isHLtitle": "false"
    }
    response = requests.post(url, data=data, headers=headers)
    return response

# 发送HTTP请求并获取响应
def download_report(date):
    global counter
    all_results = []
    page_num = 1
    response_test = get_report(page_num, date)

    try:
        data_test = response_test.json()
        total_pages = data_test.get("totalpages", 0)
        if total_pages == 0:
            return all_results
    except (ValueError, KeyError) as e:
        print(f"获取总页数失败: {e}")
        return all_results

    max_retries = 3
    while page_num <= total_pages + 1:
        retry_count = 0
        while retry_count <= max_retries:
            try:
                response = get_report(page_num, date)
                response.raise_for_status()
                data = response.json()
                if not data.get("announcements"):
                    break
                all_results.extend(data["announcements"])
                if total_pages > 0:
                    per = (counter / total_pages)
                    if per < 1:
                        print(f"\r当前年份下载进度 {per * 100:.2f} %", end='')
                    else:
                        print(f"\r下载完成，正在保存……", end='')
                else:
                    print("无法计算下载进度，总页数为0。")
                break
            except requests.exceptions.RequestException as e:
                print(f"出现网络请求错误！: {e}")
                print("5秒后重试...")
                time.sleep(5)
                retry_count += 1
            except (ValueError, KeyError) as e:
                print(f"解析响应数据失败: {e}")
                print("5秒后重试...")
                time.sleep(5)
                retry_count += 1
            if retry_count > max_retries:
                print(f"{max_retries}次重试后均失败. 跳过第{page_num}页.")
                break
        page_num += 1
        counter += 1
    return all_results

def main(year):
    global sum
    date_count = f"{year}-01-01~{year}-12-31"
    response = get_report(1, date_count)
    data = response.json()
    sum = data.get("totalpages", 0)

    year = year + 1
    all_results = []
    time_segments = [
        f"{year}-01-01~{year}-04-01",
        f"{year}-04-02~{year}-04-15",
        f"{year}-04-16~{year}-04-22",
        f"{year}-04-23~{year}-04-26",
        f"{year}-04-27~{year}-04-28",
        f"{year}-04-29~{year}-04-30",
        f"{year}-05-01~{year}-07-31",
        f"{year}-08-01~{year}-10-31",
        f"{year}-11-01~{year}-11-30",
        f"{year}-12-01~{year}-12-31"
    ]
    for segment in time_segments:
        results = download_report(segment)
        all_results.extend(results)

    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    worksheet.title = "年报"
    worksheet.append(["公司代码", "公司简称", "标题", "年份", "年报链接"])

    for item in all_results:
        company_code = item["secCode"]
        company_name = item["secName"]
        title = item["announcementTitle"].strip()
        title = re.sub(r"<.*?>", "", title)
        title = title.replace("：", "")
        title = f"《{title}》"
        adjunct_url = item["adjunctUrl"]
        m = re.search(r"(\d{4})年", title)
        year_str = m.group(1) if m else setYear
        announcement_url = f"http://static.cninfo.com.cn/{adjunct_url}"
        exclude_flag = any(kw in title for kw in exclude_keywords)
        if not exclude_flag:
            worksheet.append([company_code, company_name, title, year_str, announcement_url])

    save_path = os.path.join(output_dir, f"年报链接_{setYear}{GZH}.xlsx")
    workbook.save(save_path)
    print(f"已保存到 {save_path}")


if __name__ == '__main__':
    # 排除列表可以加入'更正后','修订版'来规避数据重复或公司发布之前年份的年报修订版等问题，
    exclude_keywords = ['英文','已取消','摘要']
    # 控制行业，若为空则不控制，仅可从参考内容中选取，中间用英文分号隔开
    # 参考内容："农、林、牧、渔业;电力、热力、燃气及水生产和供应业;建筑业;采矿业;制造业;批发和零售业;交通运输、仓储和邮政业;住宿和餐饮业;信息传输、软件和信息技术服务业;金融业;房地产业;租赁和商务服务业;科学研究和技术服务业;水利、环境和公共设施管理业;居民服务、修理和其他服务业;教育;卫生和社会工作;文化、体育和娱乐业;综合"
    trade = ""
    # 板块控制：深市sz 沪市sh 深主板szmb 沪主板shmb 创业板szcy 科创板shkcp 北交所bj 请按照格式填写
    plate = "sz;sh"
    # ========= 可选设置：自定义公司列表，仅下载指定公司年报 =========
    # 示例：['600519', '000001']，留空则下载全部
    secCode = ['300500','000670']
    # 示例：['中集集团', '中洲控股']，留空则下载全部
    secName = []


    global counter
    global sum
    counter = 1
    setYear = 2024
    Flag = 0

    def filter_excel(year):
        if not secCode and not secName:
            return
        try:
            src_path = os.path.join(output_dir, f"年报链接_{year}{GZH}.xlsx")
            wb = openpyxl.load_workbook(src_path)
            ws = wb.active
            rows = list(ws.iter_rows(values_only=True))
            header = rows[0]
            filtered = [header] + [r for r in rows[1:]
                                   if str(r[0]) in secCode or str(r[1]) in secName]
            wb_new = openpyxl.Workbook()
            ws_new = wb_new.active
            for r in filtered:
                ws_new.append(r)
            dst_path = os.path.join(output_dir, f"年报链接_{year}_选取公司.xlsx")
            wb_new.save(dst_path)
            print(f"---- 过滤完成: {dst_path} 已生成")
        except Exception as e:
            print(f"---- 过滤失败: {e}，已跳过")

    if Flag:
        for y in range(2020, setYear + 1):
            counter = 1
            main(y)
            filter_excel(y)
            print(f"---- {y}年下载完成")
    else:
        main(setYear)
        filter_excel(setYear)
        print(f"---- {setYear}年下载完成")



下载完成，正在保存…….57 %已保存到 /Users/sam/Desktop/cninfo_output/年报链接_2024【年报】.xlsx
---- 过滤完成: /Users/sam/Desktop/cninfo_output/年报链接_2024_选取公司.xlsx 已生成
---- 2024年下载完成


In [18]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
'''
@Project ：PycharmProjects
@File    ：巨潮资讯年报2.0.py
@IDE     ：PyCharm
@Author  ：lingxiaotian
@Date    ：2023/5/20 12:38
'''

import requests
import re
import openpyxl
import time
import os

# 放到桌面上的 cninfo_output 文件夹
output_dir = os.path.expanduser("~/Desktop/cninfo_output")
os.makedirs(output_dir, exist_ok=True)

GZH = "【年报】"

def get_report(page_num, date):
    url = "http://www.cninfo.com.cn/new/hisAnnouncement/query"
    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
        "Content-Length": "195",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Host": "www.cninfo.com.cn",
        "Origin": "http://www.cninfo.com.cn",
        "Proxy-Connection": "keep-alive",
        "Referer": "http://www.cninfo.com.cn/new/commonUrl/pageOfSearch?url=disclosure/list/search&checkedCategory=category_ndbg_szsh",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42",
        "X-Requested-With": "XMLHttpRequest"
    }
    data = {
        "pageNum": page_num,
        "pageSize": 30,
        "column": "szse",
        "tabName": "fulltext",
        "plate": plate,
        "searchkey": "",
        "secid": "",
        "category": "category_ndbg_szsh",
        "trade": trade,
        "seDate": date,
        "sortName": "code",
        "sortType": "asc",
        "isHLtitle": "false"
    }
    return requests.post(url, data=data, headers=headers)


def download_report(date):
    global counter
    all_results = []
    page_num = 1
    response_test = get_report(page_num, date)

    try:
        data_test = response_test.json()
        total_pages = data_test.get("totalpages", 0)
        if total_pages == 0:
            return all_results
    except (ValueError, KeyError) as e:
        print(f"获取总页数失败: {e}")
        return all_results

    max_retries = 3
    while page_num <= total_pages + 1:
        retry_count = 0
        while retry_count <= max_retries:
            try:
                response = get_report(page_num, date)
                response.raise_for_status()
                data = response.json()
                if not data.get("announcements"):
                    break
                all_results.extend(data["announcements"])
                if total_pages > 0:
                    per = (counter / total_pages)
                    if per < 1:
                        print(f"\r当前年份下载进度 {per * 100:.2f}%", end='')
                    else:
                        print(f"\r下载完成，正在保存……", end='')
                else:
                    print("无法计算下载进度，总页数为0。")
                break
            except Exception as e:
                print(f"第{page_num}页错误: {e}，5秒后重试...")
                time.sleep(5)
                retry_count += 1
        page_num += 1
        counter += 1
    return all_results


def main(year):
    global sum
    date_count = f"{year}-01-01~{year}-12-31"
    response = get_report(1, date_count)
    data = response.json()
    sum = data.get("totalpages", 0)

    year += 1
    all_results = []
    time_segments = [
        f"{year}-01-01~{year}-04-01",
        f"{year}-04-02~{year}-04-15",
        f"{year}-04-16~{year}-04-22",
        f"{year}-04-23~{year}-04-26",
        f"{year}-04-27~{year}-04-28",
        f"{year}-04-29~{year}-04-30",
        f"{year}-05-01~{year}-07-31",
        f"{year}-08-01~{year}-10-31",
        f"{year}-11-01~{year}-11-30",
        f"{year}-12-01~{year}-12-31"
    ]
    for segment in time_segments:
        all_results.extend(download_report(segment))

    # ========== 可选：保留最新版本年报 ==========

    def filter_latest_versions(ans):
        keywords = ['更正','更正后','更正版','修订后','修订版','更新后','更新版']
        latest = {}
        for item in ans:
            code = item['secCode']
            m = re.search(r"(\d{4})年", item['announcementTitle'])
            yr = m.group(1) if m else ''
            key = (code, yr)
            title = item['announcementTitle']
            if key not in latest or (any(kw in title for kw in keywords) and not any(kw in latest[key]['announcementTitle'] for kw in keywords)):
                latest[key] = item
        return list(latest.values())
    # 启用下行以仅保留更新后年报
    all_results = filter_latest_versions(all_results)
    

    workbook = openpyxl.Workbook()
    ws = workbook.active
    ws.title = "年报"
    ws.append(["公司代码", "公司简称", "标题", "年份", "年报链接"])
    for item in all_results:
        company_code = item['secCode']
        company_name = item['secName']
        title = re.sub(r"<.*?>", "", item['announcementTitle'].strip()).replace('：','')
        title = f"《{title}》"
        m = re.search(r"(\d{4})年", title)
        year_str = m.group(1) if m else setYear
        url = f"http://static.cninfo.com.cn/{item['adjunctUrl']}"
        if not any(kw in title for kw in exclude_keywords):
            ws.append([company_code, company_name, title, year_str, url])

    save_path = os.path.join(output_dir, f"年报链接_{setYear}{GZH}.xlsx")
    workbook.save(save_path)
    print(f"已保存到 {save_path}")

if __name__ == '__main__':
    # 排除列表可以加入'更正后','修订版'来规避数据重复或公司发布之前年份的年报修订版等问题，
    exclude_keywords = ['英文','已取消','摘要']
    # 控制行业，若为空则不控制，仅可从参考内容中选取，中间用英文分号隔开
    # 参考内容："农、林、牧、渔业;电力、热力、燃气及水生产和供应业;建筑业;采矿业;制造业;批发和零售业;交通运输、仓储和邮政业;住宿和餐饮业;信息传输、软件和信息技术服务业;金融业;房地产业;租赁和商务服务业;科学研究和技术服务业;水利、环境和公共设施管理业;居民服务、修理和其他服务业;教育;卫生和社会工作;文化、体育和娱乐业;综合"
    trade = ""
    # 板块控制：深市sz 沪市sh 深主板szmb 沪主板shmb 创业板szcy 科创板shkcp 北交所bj 请按照格式填写
    plate = "sz;sh"
    # ========= 可选设置：自定义公司列表，仅下载指定公司年报 =========
    # 示例：['600519', '000001']，留空则下载全部
    secCode = ['300500','000670']
    # 示例：['中集集团', '中洲控股']，留空则下载全部
    secName = []

    global counter, sum
    counter = 1
    setYear = 2024
    Flag = 0

    def filter_excel(year):
        if not secCode and not secName:
            return
        try:
            src = os.path.join(output_dir, f"年报链接_{year}{GZH}.xlsx")
            wb = openpyxl.load_workbook(src)
            ws = wb.active
            rows = list(ws.iter_rows(values_only=True))
            header = rows[0]
            filtered = [header] + [r for r in rows[1:] if str(r[0]) in secCode or str(r[1]) in secName]
            wb_new = openpyxl.Workbook()
            ws_new = wb_new.active
            for r in filtered:
                ws_new.append(r)
            dst = os.path.join(output_dir, f"年报链接_{year}_选取公司.xlsx")
            wb_new.save(dst)
            print(f"过滤完成: {dst} 已生成")
        except Exception as e:
            print(f"过滤失败: {e}，已跳过")

    if Flag:
        for y in range(2020, setYear+1):
            counter = 1
            main(y)
            filter_excel(y)
            print(f"----{y}年下载完成")
    else:
        main(setYear)
        filter_excel(setYear)
        print(f"----{setYear}年下载完成")


下载完成，正在保存…….57%已保存到 /Users/sam/Desktop/cninfo_output/年报链接_2024【年报】.xlsx
过滤完成: /Users/sam/Desktop/cninfo_output/年报链接_2024_选取公司.xlsx 已生成
----2024年下载完成
